# Examen 2

Equipo 1:

* Bautista Bautista Cristian
* Madariaga Dominguez Alejandra
* Montejo Isidoro David Emmanuel.
* 

# Objetivos:

* Predicción de popularidad de canciones
* Recomendación personalizada de canciones.
* Clasificación automatica de generos musicales.

* Unidad muestral por objetivo.
um_1: song_play_counts_df, summary_df, audio_features_df, song_lyrics_df, song_genres_df

* Canciones por usuario.
um_2: song_play_counts_df, summary_df, audio_features_df, song_lyrics_df, song_genres_df


um_3: song_play_counts_df, summary_df, audio_features_df, song_genres_df, song_lyrics_df

## Dependencias.

In [236]:
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
# from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor


import nltk
from nltk.corpus import stopwords

import joblib



## Funciones.

## Carga de datos

In [237]:
audio_features_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/audio_features.csv')
print('audio_features_df', audio_features_df.shape)

song_play_counts_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/song_play_count.csv')
print('song_play_counts_df', song_play_counts_df.shape)

artist_terms_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/artist_terms.csv')
print('artist_terms_df', artist_terms_df.shape)

song_genres_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/song_genres.csv')
print('song_genres_df', song_genres_df.shape)

artist_location_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/artist_location.csv')
print('artist_location_df', artist_location_df.shape)

song_lyrics_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/song_lyrics.csv')
print('song_lyrics_df', song_lyrics_df.shape)

artist_tags_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/artist_tags.csv')
print('artist_tags_df', artist_tags_df.shape)

summary_df = pd.read_csv('/home/david/Documents/DIPLO/modulo_2/Examen/Examen2/datos/summary.csv')
print('summary_df', summary_df.shape)



audio_features_df (19664, 23)
song_play_counts_df (1517641, 3)
artist_terms_df (12947, 2)
song_genres_df (8416, 2)
artist_location_df (13850, 5)
song_lyrics_df (6803, 101)
artist_tags_df (15375, 2)
summary_df (20000, 20)


### Summary

In [238]:
summary_df.shape

(20000, 20)

In [239]:
df = summary_df.copy()
print(df.shape)

(20000, 20)


In [240]:
song_lyrics_df.head(1)

,track_id,a,all,am,and,are,as,at,away,babi,...,when,where,who,will,with,would,y,yeah,you,your
0,TRSZDWT128F4282FF8,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [241]:
df['track_id'].nunique()




20000

In [242]:
df['song_id'].nunique()

19997

In [243]:
registros_eliminar = df[df['song_id'].duplicated()]['track_id'].tolist()
df = df[~df['track_id'].isin(registros_eliminar)]
print(df.shape)

(19997, 20)


In [244]:
df

,track_id,song_id,mode_confidence,end_of_fade_in,key_confidence,energy,year,duration,time_signature_confidence,title,start_of_fade_out,analysis_sample_rate,time_signature,tempo,artist_id,artist_name,artist_hotttnesss,key,release,loudness
0,TRJPTGQ128F42A2187,SOIYHHS12A8C13E7B5,0.452,0.000,0.556,0.0,2007,141.66159,1.000,Don't Cry,141.662,22050,4,103.085,ARJ5FD21187FB43F12,Mavado,0.520784,0,Gangsta For Life: The Symphony Of David Brooks,-5.816
1,TRZWFET128F425BF9E,SONEJET12A8C137D3E,0.669,0.000,0.679,0.0,2005,354.69016,0.529,Gypsy Funeral: Hark! From The Tombs Of Doleful...,348.253,22050,1,123.153,ARH3CTU1187FB365C7,Miasma & The Carousel Of Headless Horses,0.346908,9,Perils,-11.137
2,TRNTAMF128F14618CA,SOPAGVA12AF729F018,0.551,0.000,0.672,0.0,2001,338.44200,1.000,Trade Money,327.529,22050,4,162.540,AR0CANF1187B9AF35F,Dilated Peoples,0.457841,7,Expansion Team,-5.199
3,TRALQGQ128F427E682,SODZLJT12A8C13BD4C,0.354,0.000,0.378,0.0,2004,202.37016,1.000,Me Han Vuelto Loca,200.220,22050,4,101.978,ARKE6XC1187B991839,Merche,0.406329,6,Autentica,-2.756
4,TRLOUXS128F92D963F,SOXWPNT12AAF3B270E,0.761,0.000,0.615,0.0,2005,315.58485,0.719,Riders On The Storm,288.142,22050,4,108.992,AR2TT8P1187B9B624D,O'2L,0.291499,9,Doyle's Brunch,-8.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,TRGWRQD128F4298D89,SOMYXHV12A8C13BA11,0.355,0.496,0.032,0.0,1998,209.16200,0.260,arm (Album Version),206.495,22050,4,90.024,AR4CO001187FB4731E,the feelers,0.486442,11,Supersystem,-6.153
19996,TRIPSRA128F931DBAA,SOHOYEM12AB017F833,0.407,0.000,0.353,0.0,2007,90.95791,0.000,Raw Diamond,90.958,22050,4,165.282,ARVRTDI1187B99D797,Baby Grandmothers,0.343689,0,Baby Grandmothers,-5.540
19997,TRSYCMI12903CE007B,SOYSKTB12A58A7B126,0.526,0.000,0.730,0.0,2005,154.67057,0.542,Boom Boom Boom East Blues,154.671,22050,4,190.921,ARCWZP81187FB40E42,Hillstomp,0.425443,2,The Woman That Ended The World,-8.800
19998,TRKVYMQ128F9303EB5,SODWMOZ12A8C13D1CE,0.409,2.682,0.749,0.0,2001,191.29424,0.767,Serenata,177.308,22050,5,139.339,ARJMB691187FB54124,Luciano Pavarotti,0.496486,7,A Song to Remember,-17.083


### soung_play_counts_df

In [245]:
print(song_play_counts_df.shape)
song_play_counts_df.head(1)

(1517641, 3)


,user_id,song_id,play_count
0,60187,SOYMXMF12A6D4FCDA2,1


In [246]:
song_play_counts_df['song_id'].nunique()

10370

In [247]:
# Nivel usuario.
song_play_counts_df.groupby('user_id', as_index=False)['play_count'].sum()

,user_id,play_count
0,0,4
1,1,5
2,2,1
3,3,2
4,4,3
...,...,...
610647,610647,13
610648,610648,1
610649,610649,19
610650,610650,1


In [248]:
song_play_counts_df

,user_id,song_id,play_count
0,60187,SOYMXMF12A6D4FCDA2,1
1,11193,SOIYNJG12AB01810B2,2
2,67953,SOYDTRQ12AF72A3D61,5
3,132173,SOLGFKA12A6D4F7B9A,1
4,578451,SOTDPWU12A8AE47B2B,4
...,...,...,...
1517636,129566,SOIPSLZ12A8C140561,1
1517637,101507,SOCIDVO12A58A7DDA4,2
1517638,573589,SOGXDRM12A6D4F7CE0,1
1517639,469534,SOIDDNJ12AC9071B90,1


In [249]:
song_play_count_grouped = song_play_counts_df.groupby('song_id').agg({'play_count': 'sum', 'user_id': 'nunique', 'user_id': list}).reset_index()
song_play_count_grouped['conteo_usuario'] = song_play_count_grouped['user_id'].apply(lambda x: len(x))
song_play_count_grouped

,song_id,play_count,user_id,conteo_usuario
0,SOAABDH12A6D4F7658,9,"[549734, 251675, 183315, 19290, 104030]",5
1,SOAAGDJ12A8C13A1C6,661,"[539095, 275887, 498098, 490329, 325807, 55017...",255
2,SOAAGQU12AB01843A8,1,[471243],1
3,SOAALEG12A58A79808,282,"[453881, 339932, 546841, 78898, 558351, 551499...",151
4,SOAAMTH12AF729D4E5,1021,"[384763, 396183, 79891, 421517, 110612, 279645...",398
...,...,...,...,...
10365,SOZZTQU12AB017FAEE,159,"[452153, 181201, 441913, 16286, 366490, 458007...",54
10366,SOZZWHT12A8C142264,125,"[324851, 27851, 530192, 195775, 179055, 248676...",48
10367,SOZZXQJ12AB0184460,4,"[378241, 128467, 131013, 340622]",4
10368,SOZZYDH12AAF3B31AF,7,"[565110, 404761]",2


In [250]:
song_play_count_grouped['song_id'].nunique()

10370

In [251]:
df.shape

(19997, 20)

In [252]:
df = df.merge(song_play_count_grouped[['song_id', 'play_count', 'conteo_usuario']], how='left', on='song_id')

In [253]:
song_play_count_grouped[song_play_count_grouped['song_id'] == 'SONEJET12A8C137D3E']

,song_id,play_count,user_id,conteo_usuario


In [254]:
df['play_count'].isna().sum() / df.shape[0]

0.4814222133319998

### song_genres_df

In [255]:
song_genres_df

,track_id,main_genre
0,TRSZBVQ128F425D5B6,Rock
1,TRRJNCE128F9319E52,RnB
2,TRIZJQX128F1472163,Folk
3,TRBXBWB128F92F9F2B,Rap
4,TRFBZFH12903CBDABB,Electronic
...,...,...
8411,TRHKNDN128F9320924,Blues
8412,TRGIYIR128F92DCDF1,Rock
8413,TRVFMDX128F42915F2,Punk
8414,TRLWOAY128F933A6A3,Rock


In [256]:
song_genres_df['track_id'].nunique()

8416

In [257]:
df = df.merge(song_genres_df, how='left', on='track_id')

### audio_features_df.

In [258]:
print(audio_features_df.shape)
audio_features_df['track_id'].nunique()

(19664, 23)


19664

In [259]:
df = df.merge(audio_features_df, how='left', on='track_id')
print(df.shape)
df.head(5)

(19997, 45)


,track_id,song_id,mode_confidence,end_of_fade_in,key_confidence,energy,year,duration,time_signature_confidence,title,...,acousticness,aggressiveness,electronicity,happiness,festiveness,relaxation,sadness,brightness,tonality,insrumentalness
0,TRJPTGQ128F42A2187,SOIYHHS12A8C13E7B5,0.452,0.0,0.556,0.0,2007,141.66159,1.000,Don't Cry,...,0.022293,0.500000,0.882711,0.456452,0.772325,0.508476,0.067847,0.878608,0.045879,0.007640
1,TRZWFET128F425BF9E,SONEJET12A8C137D3E,0.669,0.0,0.679,0.0,2005,354.69016,0.529,Gypsy Funeral: Hark! From The Tombs Of Doleful...,...,0.003386,0.918240,0.395611,0.110496,0.518365,0.262771,0.067540,0.278791,0.150650,0.798211
2,TRNTAMF128F14618CA,SOPAGVA12AF729F018,0.551,0.0,0.672,0.0,2001,338.44200,1.000,Trade Money,...,0.028752,0.471321,0.934747,0.484657,0.726565,0.012293,0.242436,0.945652,0.123880,0.315464
3,TRALQGQ128F427E682,SODZLJT12A8C13BD4C,0.354,0.0,0.378,0.0,2004,202.37016,1.000,Me Han Vuelto Loca,...,0.037619,0.242321,0.899101,0.500000,0.860772,0.113721,0.437496,0.776979,0.784130,0.080626
4,TRLOUXS128F92D963F,SOXWPNT12AAF3B270E,0.761,0.0,0.615,0.0,2005,315.58485,0.719,Riders On The Storm,...,0.463796,0.071169,0.794642,0.339365,0.082585,0.974410,0.669444,0.276611,0.030401,0.676081


### song_lyrics_df

In [260]:
song_lyrics_df.head()

,track_id,a,all,am,and,are,as,at,away,babi,...,when,where,who,will,with,would,y,yeah,you,your
0,TRSZDWT128F4282FF8,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRFVRRO128F426294C,2.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,9.0,3.0,1.0,0.0,0.0,8.0,0.0
2,TRRCNRA128F9348098,1.0,1.0,0.0,1.0,5.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,16.0,4.0
3,TRMWJFM128F4235893,4.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,0.0,...,2.0,6.0,1.0,1.0,2.0,2.0,0.0,0.0,11.0,18.0
4,TRBMXWS12903CDEC1A,4.0,2.0,0.0,8.0,2.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [261]:
print(song_lyrics_df.shape)
print(song_lyrics_df['track_id'].nunique())

(6803, 101)
6803


In [262]:
df = df.merge(song_lyrics_df, how='left', on='track_id')

### Target

In [263]:
df['play_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.99])

count    10370.000000
mean       407.235391
std       2149.946124
min          1.000000
25%         10.000000
50%         41.000000
75%        170.000000
80%        252.000000
90%        690.300000
95%       1528.550000
99%       6547.200000
max      83255.000000
Name: play_count, dtype: float64

In [264]:
percentiles = df['play_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.99])
print(percentiles)

count    10370.000000
mean       407.235391
std       2149.946124
min          1.000000
25%         10.000000
50%         41.000000
75%        170.000000
80%        252.000000
90%        690.300000
95%       1528.550000
99%       6547.200000
max      83255.000000
Name: play_count, dtype: float64


In [265]:
bins = [df['play_count'].min(), percentiles['25%'], percentiles['50%'], percentiles['75%'], percentiles['80%'], percentiles['90%'], percentiles['95%'], percentiles['99%'], df['play_count'].max()]
labels = ['< 25%', '25%-50%', '50%-75%', '75%-80%', '80%-90%', '90%-95%', '95%-99%', '> 99%']
df['play_count_interval'] = pd.cut(df['play_count'], bins=bins, labels=labels, include_lowest=True)


In [266]:
df['play_count_interval'].value_counts(dropna=False)

play_count_interval
NaN        9627
< 25%      2598
25%-50%    2594
50%-75%    2586
80%-90%    1031
75%-80%     524
90%-95%     518
95%-99%     415
> 99%       104
Name: count, dtype: int64

In [267]:
# Eliminar esta columna.
df = df.drop(['play_count_interval'], axis= 1)
print(df.shape)

(19997, 145)


In [268]:
# Target de clasificación.

percentil = df['play_count'].quantile(0.75)
df['target'] = (df['play_count'] > percentil).astype(int)
print(df['target'].value_counts(normalize=True))

target
0    0.870381
1    0.129619
Name: proportion, dtype: float64


In [269]:
df_valid = df[df['play_count'].isna()]
print(df_valid.shape)

(9627, 146)


In [271]:
df_train = df[df['play_count'].notna()]
print(df_train.shape)


(10370, 146)


## Limpieza de datos

In [272]:
df_train.isna().sum()[df_train.isna().sum() >0]

main_genre                6462
analysis_sample_rate_y    1498
bit_rate                  1498
equal_loudness            1498
length                    1498
                          ... 
would                     5895
y                         5895
yeah                      5895
you                       5895
your                      5895
Length: 123, dtype: int64

## Ingeniería de características.

In [ ]:
df_train.shape

(10370, 146)

In [ ]:
col_texto = df_train.columns[df_train.dtypes == 'object'].tolist()
print(len(col_texto))


10


In [ ]:
df_train.filter(like='targ')

,c_target
0,0
2,0
3,0
5,0
6,0
...,...
19987,0
19991,0
19992,0
19994,0


In [273]:
col_continuas = (df_train.columns[(df_train.dtypes == 'float') | (df_train.dtypes == 'int')]).tolist()
col_continuas.remove('target')
print(len(col_continuas))

135


In [274]:
df_train.rename(columns={col: 'c_' + col for col in col_continuas}, inplace=True)

/tmp/ipykernel_2240/2698531862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.rename(columns={col: 'c_' + col for col in col_continuas}, inplace=True)


In [278]:
## df con varaibles continuas y discretas 
varc         = [v for v in df_train.columns if v[:2]=='c_']
df_tad = df_train[varc + ['target']].copy()
print(df_tad.shape)

(10370, 136)


In [281]:
df_tad = df_tad.dropna()
print(df_tad.shape)


(3850, 136)


## Modelación.

In [282]:
X = df_tad.drop(columns=['target'])
print(X.shape)
y = df_tad['target']

(3850, 135)


In [283]:
X[varc]

,c_mode_confidence,c_end_of_fade_in,c_key_confidence,c_energy,c_year,c_duration,c_time_signature_confidence,c_start_of_fade_out,c_analysis_sample_rate_x,c_time_signature,...,c_when,c_where,c_who,c_will,c_with,c_would,c_y,c_yeah,c_you,c_your
0,0.452,0.000,0.556,0.0,2007,141.66159,1.000,141.662,22050,4,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,32.0,6.0
2,0.551,0.000,0.672,0.0,2001,338.44200,1.000,327.529,22050,4,...,1.0,1.0,2.0,1.0,4.0,1.0,0.0,0.0,6.0,3.0
3,0.354,0.000,0.378,0.0,2004,202.37016,1.000,200.220,22050,4,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
10,0.512,2.125,0.330,0.0,2007,220.70812,1.000,206.965,22050,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
20,0.461,0.113,0.601,0.0,2007,196.10077,0.886,191.843,22050,5,...,1.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,10.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19951,0.557,0.165,0.713,0.0,2005,250.33098,0.939,235.764,22050,4,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,16.0
19968,0.293,0.084,0.146,0.0,2002,307.87873,0.596,274.175,22050,4,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,2.0,2.0
19979,0.695,0.305,0.702,0.0,2008,255.50322,1.000,239.299,22050,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19985,0.619,0.728,0.684,0.0,2004,218.06975,0.398,202.263,22050,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0


In [284]:
sc_x = MinMaxScaler()
joblib.dump(sc_x, 'scaler_min_max_clasificacion.pkl')

X[varc] = sc_x.fit_transform(X[varc])

In [286]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)

In [287]:
# Importar regresión logística.
from sklearn.linear_model import LogisticRegression

In [288]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [ ]:
# predecir en test
y_pred = lr.predict(X_test)



In [289]:
ls_scores = cross_val_score(X=X_test, y=y_test, cv=4, n_jobs=2, estimator=lr, scoring="roc_auc")

In [290]:
ls_scores

array([0.7194709 , 0.72169312, 0.65740741, 0.77898936])

In [291]:
np.mean(ls_scores), np.std(ls_scores)

(0.7193901975683891, 0.04300940773220997)

## Resultados

* 